# 🎨 Data Designer 101: Seeding Synthetic Data Generation with an External Dataset

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/3-seeding-with-a-dataset.ipynb) to download this notebook to your computer.

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) are used.


In [2]:
data_designer_client = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[10:18:54] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder)

[10:18:54] [INFO] 🧐 Preview generation in progress


[10:18:54] [INFO] ✅ Validation passed


[10:18:54] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[10:18:54] [INFO] 🩺 Running health checks for models...


[10:18:54] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...


...

[10:19:17] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[10:19:17] [INFO]   |-- 🎲 column: 'date_of_visit'


[10:19:17] [INFO]   |-- 🧩 column: 'physician'


[10:19:17] [INFO]   |-- 📝 column: 'physician_notes'


[10:19:17] [INFO] 🎊 Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': 'd05951f9-fbf3-4756-b0da-31a15114a4fc',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Andrea',                                                                │
│                    │     'last_name': 'Figueroa',                                                               │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Female',                                                                       │
│                    │     'street_number': '1586',                                                               │
│                    │     'street_name': 'Sara Prairie',                                                         │
│                    │     'city': 'Matthewchester',                                                              │
│                    │     'state': 'North Carolina',                                                             │
│                    │     'postcode': '80008',                                                                   │
│                    │     'age': 68,                                                                             │
│                    │     'birth_date': '1957-01-01',                                                            │
│                    │     'country': 'Brunei Darussalam',                                                        │
│                    │     'marital_status': 'never_married',                                                     │
│                    │     'education_level': 'associates',                                                       │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'Chemical engineer',                                                     │
│                    │     'phone_number': '(685)212-809

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': 'd05951f9-fbf3-4756-b0da-31a15114a4fc...,{'uuid': 'b29cfc6b-649f-43eb-9bc6-bb4dcbeb4594...,PT-E2359657,2024-07-17,2024-07-25,Andrea,Figueroa,1957-01-01,Dr. Huerta,"**Dr. Zoe Huerta, MD** \n**Primary Care Physi..."
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': '0288aeec-85c9-40c8-9f1d-ae5ade2af1b6...,{'uuid': 'b27da88e-1302-4441-abf0-b7ef1514c80e...,PT-30903280,2024-03-04,2024-03-09,Justin,Rodriguez,1929-09-29,Dr. Gray,**Patient Name:** Justin Rodriguez \n**Date o...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,{'uuid': '9777e48c-4f40-462a-a46d-03dc8ac92712...,{'uuid': 'dcbfc56d-f31f-4d40-9103-314fd73a03b8...,PT-69701D3F,2024-06-03,2024-06-22,Tracey,Allen,2005-06-22,Dr. Mccormick,**Dr. James Mccormick** \n**Primary Care Phys...
3,arthritis,I have been having trouble with my muscles and...,{'uuid': 'c93d1038-16f4-4ec8-abda-fb8f0d3fb676...,{'uuid': '9a7c4a41-1bf6-4e91-9fc8-335de79fe1e5...,PT-9BAA33F9,2024-12-28,2025-01-05,Madison,Rodriguez,1999-09-01,Dr. Ross,**Dr. John Ross** \n**Primary Care Physician*...
4,dengue,I have been feeling really sick. My body hurts...,{'uuid': '551f3a12-b2a4-41b1-80c4-d0a2f696bdf9...,{'uuid': 'c4374cad-34c1-4556-88ba-0bb8e986f64e...,PT-04015F9C,2024-03-28,2024-04-21,Jackie,Black,1922-06-07,Dr. Wheeler,**Dr. Kenneth Wheeler** \n**Primary Care Phys...
5,common cold,I've been feeling really run down and weak. My...,{'uuid': 'd8eb090d-84b1-4e2e-a833-12e8a70ae7a5...,{'uuid': 'fe995adb-ea95-48cc-80c4-c01520fa7b11...,PT-47220B6B,2024-02-26,2024-03-10,Brenda,Collins,1975-09-01,Dr. Williams,**Patient Name:** Brenda Collins \n**Date of ...
6,dengue,I have rashes all over my body. I also have a ...,{'uuid': '5744ca48-6238-4e60-9ad7-fe87f7d9ea4f...,{'uuid': '8a5f1399-00fb-4436-9ee7-fba763b28f79...,PT-ED3BD82B,2024-11-08,2024-11-20,Renee,Lyons,1922-08-05,Dr. Brown,**Dr. Christopher Brown** \n**Primary Care Ph...
7,dengue,I have been feeling nauseous and have a consta...,{'uuid': 'a4667b82-1366-477a-b0d1-00645e4fb82e...,{'uuid': '4f735ccb-1d3b-4f8e-bbba-fa19ae078793...,PT-4034E22E,2024-11-11,2024-11-14,Debbie,Allen,1922-11-13,Dr. Blankenship,"**Dr. Patricia Blankenship, MD** \n**Primary ..."
8,impetigo,I have a rash around my nose that is red and i...,{'uuid': 'eece44e9-246c-4c03-8294-1665dfb31d3e...,{'uuid': '4ca33145-6071-4cff-8f30-9bd21aa8975d...,PT-F45B2D2D,2024-08-11,2024-08-31,Michael,Mcfarland,2001-06-12,Dr. Lucero,"**Dr. Joshua Lucero, Primary Care Physician** ..."
9,drug reaction,I have a rash on my chest and back and it itch...,{'uuid': '1bab800f-9df0-4396-bbde-976218ff5a18...,{'uuid': '24a57465-13aa-4a78-a3d3-2636dff06a76...,PT-BBBCF8A2,2024-10-08,2024-10-31,Rachel,Caldwell,1938-09-30,Dr. Hodges,**Dr. Antonio Hodges** \n**Primary Care Physi...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      10 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=20)

[10:19:17] [INFO] 🎨 Creating Data Designer dataset


[10:19:17] [INFO] ✅ Validation passed


[10:19:17] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[10:19:17] [INFO] 📂 Dataset path '/Users/amanoel/Code/DataDesigner/docs/notebook_source/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/Users/amanoel/Code/DataDesigner/docs/notebook_source/artifacts/dataset_12-03-2025_101917' instead.


[10:19:17] [INFO] 🩺 Running health checks for models...


...

[10:19:55] [INFO]   |-- 🧩 column: 'dob'


[10:19:55] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[10:19:55] [INFO]   |-- 🎲 column: 'date_of_visit'


[10:19:55] [INFO]   |-- 🧩 column: 'physician'


[10:19:55] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 36, 'bachelors_field': 'business', 'bi...","{'age': 55, 'bachelors_field': 'no_degree', 'b...",PT-29327BB5,2024-11-30,2024-12-26,Danielle,Bailey,1989-03-22,Dr. Mcdonald,"**Dr. Christine McDonald, MD** **Primary Car..."
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 34, 'bachelors_field': 'no_degree', 'b...","{'age': 105, 'bachelors_field': 'no_degree', '...",PT-4ED83D65,2024-08-15,2024-08-31,Kevin,Robbins,1991-03-11,Dr. Bowman,"**Dr. Tina Bowman, MD** **Primary Care Physi..."
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 99, 'bachelors_field': 'stem', 'birth_...","{'age': 94, 'bachelors_field': 'stem', 'birth_...",PT-1AF5229B,2024-01-28,2024-01-31,Austin,Lopez,1926-11-28,Dr. Bishop,**Dr. April Bishop** **Primary Care Physicia...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 79, 'bachelors_field': 'no_degree', 'b...","{'age': 48, 'bachelors_field': 'arts_humanitie...",PT-3D4545C4,2024-02-10,2024-02-25,Chelsea,Morales,1946-08-21,Dr. Ramos,**Dr. Richard Ramos** **Primary Care Physici...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 108, 'bachelors_field': 'stem', 'birth...","{'age': 77, 'bachelors_field': 'stem_related',...",PT-9B4AF89E,2024-06-30,2024-07-02,Ricky,Taylor,1917-05-08,Dr. King,**Patient Visit Notes** **Date:** 2024-07-02...


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     12 (60.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    20 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      20 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      20 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      20 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      20 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Use Data Designer to generate synthetic data for your specific use case!
